In [128]:
import os
import numpy as np
import re
import json
import pandas as pd
from PIL import Image, ImageOps, UnidentifiedImageError


In [129]:
# 데이터프레임 두개 merge
fil1=pd.read_csv('./orizin_bbox_dot.csv')
fil2=pd.read_csv('./orizin_resize.csv')

merge_df = pd.merge(fil1, fil2, on='file_name', how='left')
merge_df.to_csv('./merged_df.csv',index=False)
merge_df

,file_name,bbox_x_center_list,bbox_y_center_list,bbox_widthlist,bbox_heightlist,original_width,original_height,padding_x,padding_y,resize_width_ratio,resize_height_ratio
0,064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...,671.846035,554.169695,351.502730,463.911710,1500,1126,0,80,0.426667,0.568384
1,064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...,697.063165,523.735260,375.155070,488.236760,1500,1126,0,80,0.426667,0.568384
2,064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...,815.846253,625.729679,339.455815,487.051536,1500,1126,0,80,0.426667,0.568384
3,064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...,624.850480,556.412900,342.588100,437.530600,1500,1126,0,80,0.426667,0.568384
4,064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...,721.000370,511.033335,371.196660,512.683210,1500,1126,0,80,0.426667,0.568384
...,...,...,...,...,...,...,...,...,...,...,...
2285,fe5faafa4983ec70a2a46c3d86f725a131f59fb7e9f898...,1659.326866,678.080215,626.147596,768.201570,3264,1836,0,140,0.196078,0.348584
2286,fe5faafa4983ec70a2a46c3d86f725a131f59fb7e9f898...,1472.689503,684.489316,629.048391,768.262369,3264,1836,0,140,0.196078,0.348584
2287,fe5faafa4983ec70a2a46c3d86f725a131f59fb7e9f898...,1571.361450,712.971300,682.706300,855.512200,3264,1836,0,140,0.196078,0.348584
2288,fe5faafa4983ec70a2a46c3d86f725a131f59fb7e9f898...,1412.549750,740.355000,658.967900,873.538800,3264,1836,0,140,0.196078,0.348584


In [130]:
for index, row in merge_df.iterrows():
    resize_x_center = (row['bbox_x_center_list']) * row['resize_width_ratio']+row['padding_x']
    resize_y_center = row['bbox_y_center_list'] * row['resize_width_ratio']  +row['padding_y']
    resize_width = row['bbox_widthlist'] * row['resize_width_ratio']
    resize_height = row['bbox_heightlist'] * row['resize_width_ratio']
    
    merge_df.loc[index, 'resize_x_center'] = resize_x_center
    merge_df.loc[index, 'resize_y_center'] = resize_y_center
    merge_df.loc[index, 'resize_width'] = resize_width
    merge_df.loc[index, 'resize_height'] = resize_height


In [131]:
resized_df=merge_df[['file_name','resize_x_center','resize_y_center','resize_width','resize_height']]
resized_df.to_csv('./resized_df.csv',index=False)
resized_df


,file_name,resize_x_center,resize_y_center,resize_width,resize_height
0,064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...,286.654308,316.445737,149.974498,197.935663
1,064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...,297.413617,303.460378,160.066163,208.314351
2,064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...,348.094401,346.977996,144.834481,207.808655
3,064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...,266.602871,317.402837,146.170923,186.679723
4,064c9650db76c75f0d8cdc31898eb031aece3cd1e78255...,307.626825,298.040890,158.377242,218.744836
...,...,...,...,...,...
2285,fe5faafa4983ec70a2a46c3d86f725a131f59fb7e9f898...,325.358209,272.956905,122.774038,150.627759
2286,fe5faafa4983ec70a2a46c3d86f725a131f59fb7e9f898...,288.762648,274.213591,123.342822,150.639680
2287,fe5faafa4983ec70a2a46c3d86f725a131f59fb7e9f898...,308.110088,279.798294,133.863980,167.747490
2288,fe5faafa4983ec70a2a46c3d86f725a131f59fb7e9f898...,276.970539,285.167647,129.209392,171.282118


In [132]:
import os
from PIL import Image, ImageOps
import cv2

main_dir = '.\\수정본'
emo_list = os.listdir(main_dir)

for emo in emo_list: # 감성 리스트
    file_folder_path = os.path.join(main_dir, emo) # main_dir + 감성 
    print(emo)

    file_list = os.listdir(file_folder_path)
    for num,file in enumerate(file_list):
        file_path = os.path.join(file_folder_path, file)
        if os.path.exists(file_path): # 파일이 확인되면
            try:
                origin_img = Image.open(file_path) # 파일을 가져와서
                img = ImageOps.exif_transpose(origin_img)

                bounding_box = resized_df[resized_df['file_name'] == file]

                print(file)
                x0 = bounding_box['resize_x_center'].values[0] - bounding_box['resize_width'].values[0] / 2
                x1 = bounding_box['resize_x_center'].values[0] + bounding_box['resize_width'].values[0] / 2
                y0 = bounding_box['resize_y_center'].values[0] - bounding_box['resize_height'].values[0] / 2
                y1 = bounding_box['resize_y_center'].values[0] + bounding_box['resize_height'].values[0] / 2
                
          
                print('x0=',x0, x1, 'y0=',y0, y1)
                start_point = (int(x0), int(y0))
                end_point = (int(x1), int(y1))
                image = cv2.rectangle(np.array(img), start_point, end_point, color=(0, 255, 0), thickness=1)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                cv2.imwrite(f"./box/{num}.jpg", image)

            except (OSError, UnidentifiedImageError) as e: # 이미지 손상 등 에러가 있으면 넘어갑니다.
                if 'image file is truncated' in str(e):
                    print(f"Error loading image: {file_path}. Skipping to the next image.")
                    continue
                else:
                    raise e


기쁨
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_공공시설&종교&의료시설_20201206203554-001-008.jpg
x0= 211.66705919999998 361.6415573333333 y0= 217.47790506666672 415.41356800000005
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_상업시설&점포&시장_20201206203613-003-034.jpg
x0= 217.38053546666666 377.44669866666675 y0= 199.30320213333331 407.6175530666667
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_상업시설&점포&시장_20201206203613-003-039.jpg
x0= 275.6771605941119 420.5116418527225 y0= 243.07366859496548 450.8823238122028
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_상업시설&점포&시장_20201206203613-003-044.jpg
x0= 193.51741013333339 339.68833280000007 y0= 224.06297600000005 410.7426986666667
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_숙박 및 거주공간_20201206203716-010-004.jpg
x0= 228.4382037333333 386.81544533333334 y0= 188.6684714666667 407.41330773333334
064c9650db76c75f0d8cdc31898eb

KeyboardInterrupt: 

In [ ]:
matching_rows = resized_df[resized_df['file_name'] == file]
aa=matching_rows['resize_x_center'].values[0]
bounding_box=resized_df.iloc[num]
bounding_box

file_name          67c256e3c79a80d90781381ec96500c1c3e4a488dfa0f2...
resize_x_center                                           246.973884
resize_y_center                                           465.808237
resize_width                                              114.241023
resize_height                                             159.709892
Name: 231, dtype: object

In [ ]:
bounding_box['resize_width'].values[0]

158.41787733333337

In [ ]:
import os
from PIL import Image, ImageOps
import cv2
import time

main_dir = '.\\수정본'
emo_list = os.listdir(main_dir)

for emo in emo_list: # 감성 리스트
    file_folder_path = os.path.join(main_dir, emo) # main_dir + 감성 
    print(emo)

    file_list = os.listdir(file_folder_path)
    for num,file in enumerate(file_list):
        file_path = os.path.join(file_folder_path, file)
        if os.path.exists(file_path): # 파일이 확인되면
        
            origin_img = Image.open(file_path) # 파일을 가져와서
            img = ImageOps.exif_transpose(origin_img)

                # 여기서 순서 안바뀌나 확인할것
            bounding_box = resized_df[resized_df['file_name'] == file]

            print(file)
            x0 = bounding_box['resize_x_center'].values[0] - bounding_box['resize_width'].values[0] / 2
            x1 = bounding_box['resize_x_center'].values[0] + bounding_box['resize_width'].values[0] / 2
            y0 = bounding_box['resize_y_center'].values[0] - bounding_box['resize_height'].values[0] / 2
            y1 = bounding_box['resize_y_center'].values[0] + bounding_box['resize_height'].values[0] / 2
            

            start_point = (int(x0), int(y0))
            end_point = (int(x1), int(y1))
            image = cv2.rectangle(np.array(img), start_point, end_point, color=(0, 255, 0), thickness=1)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            cv2.imwrite(f"./box/{num}.jpg", image)
            time.sleep(0.5)



기쁨
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_공공시설&종교&의료시설_20201206203554-001-008.jpg
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_상업시설&점포&시장_20201206203613-003-034.jpg
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_상업시설&점포&시장_20201206203613-003-039.jpg
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_상업시설&점포&시장_20201206203613-003-044.jpg
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_숙박 및 거주공간_20201206203716-010-004.jpg
064c9650db76c75f0d8cdc31898eb031aece3cd1e7825531d841730daefade73_여_20_기쁨_숙박 및 거주공간_20201206203716-010-006.jpg
088fed93316d43566be2fd844159eabf77e3025bf465076921ef38e9726bd8c1_남_20_기쁨_도심 환경_20201209150257-008-023.jpg
088fed93316d43566be2fd844159eabf77e3025bf465076921ef38e9726bd8c1_남_20_기쁨_숙박 및 거주공간_20201209150343-010-001.jpg
088fed93316d43566be2fd844159eabf77e3025bf465076921ef38e9726bd8c1_남_20_기쁨_숙박 및 거주공간_20201209150343-010-002.jpg
088fe

KeyboardInterrupt: 

189.96454238397382